## Deliverable 2. Create a Customer Travel Destinations Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key 

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [24]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Mount Isa,AU,-20.7333,139.5000,84.20,21,14,16.11,few clouds,2021-04-11 03:11:31
1,1,Mitsamiouli,KM,-11.3847,43.2844,81.32,76,94,8.81,light rain,2021-04-11 03:11:32
2,2,Geraldton,AU,-28.7667,114.6000,89.60,33,85,25.32,overcast clouds,2021-04-11 03:11:32
3,3,Tiksi,RU,71.6872,128.8694,0.00,81,21,7.70,few clouds,2021-04-11 03:11:33
4,4,Namibe,AO,-15.1961,12.1522,79.14,69,45,5.61,scattered clouds,2021-04-11 03:06:36


In [25]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


In [27]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)



ValueError: Can only compare identically-labeled Series objects

In [28]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()


City_ID                109
City                   109
Country                109
Lat                    109
Lng                    109
Max Temp               109
Humidity               109
Cloudiness             109
Wind Speed             109
Current Description    109
Date                   109
dtype: int64

In [29]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()


In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mount Isa,AU,84.20,few clouds,-20.7333,139.5000,
1,Mitsamiouli,KM,81.32,light rain,-11.3847,43.2844,
2,Geraldton,AU,89.60,overcast clouds,-28.7667,114.6000,
11,Santa Marta,CO,82.00,few clouds,11.2408,-74.1990,
13,Twante,MM,86.00,broken clouds,16.7167,95.9333,
21,Alofi,NU,82.40,clear sky,-19.0595,-169.9187,
25,Tessalit,ML,82.13,clear sky,20.1986,1.0114,
28,Kaduqli,SD,80.74,overcast clouds,11.0167,29.7167,
35,Hithadhoo,MV,83.34,overcast clouds,-0.6000,73.0833,
40,Carnarvon,AU,86.00,light rain,-24.8667,113.6333,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params['location'] = f'{lat},{lng}'

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 

    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")

    print(hotels)      

hment'], 'user_ratings_total': 20, 'vicinity': 'National Hiway, Santa Rosa'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 14.3240912, 'lng': 121.1130774}, 'viewport': {'northeast': {'lat': 14.3254516302915, 'lng': 121.1141172302915}, 'southwest': {'lat': 14.3227536697085, 'lng': 121.1114192697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Southpick Resort', 'photos': [{'height': 3456, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108306599661831573418">seisan sama</a>'], 'photo_reference': 'ATtYBwKNX71siYj8WngK7DejN1P62ugqSvlWgenNSOis3r2FVjw1zxYDZ1nbUAhinBS1GRQN7cRInKdlNafDyaU3Xz4wGUsOsl7b9TNg0zRwnobCiwbGHAXak0aJq9CYlZJyGkwm7WSH4c_yy__Rx2Vtwys7CuJ33eT8dsRPKqIVW_XOyeGD', 'width': 4608}], 'place_id': 'ChIJd6JcjZfZlzMRxh3eUBM2IPg', 'plus_code': {'compound_code': '84F7+J6 Santa Rosa, Laguna, Philippines', 'global_code': '7Q6384F7+J6'}, 'rating': 3.8, 'reference': 'ChIJd6JcjZfZlzMRxh3eUBM2

In [32]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna()


In [33]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 

info_box_content = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Store the DataFrame Row.
hotel_info = [info_box_content.format(**row) for index, row in hotel_df.iterrows()]

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))